<a href="https://colab.research.google.com/github/dinhtruongng/VNese-Aspect-based-Sentiment-Analysis/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [2]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer 
from skmultilearn.adapt import MLkNN 
from sklearn.metrics import hamming_loss, accuracy_score 

In [4]:
train_df = pd.read_csv('Train.csv')
dev_df = pd.read_csv("Dev.csv")
test_df = pd.read_csv("Test.csv")

In [8]:
print(train_df)

      index                                            comment  n_star  \
0         0  Mới mua máy này Tại thegioididong thốt nốt cảm...       5   
1         1  Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...       5   
2         2  Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...       3   
3         3  Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...       3   
4         4  Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...       5   
...     ...                                                ...     ...   
7781   7781  8g. Cái đi đánh là mạng giật giật ko chịu nổi....       1   
7782   7782  Mua dk giảm 500k mà lỗi lòi ra hết treo màn hì...       1   
7783   7783  Máy Sài 3 tháng rồi rất OK.pin trâu khỏi nói S...       5   
7784   7784  Rất tiếc hàng realme ko có ốp lưng ngoài , nên...       3   
7785   7785  Mình rất thất vọng khi mua máy này. Bắt wifi c...       1   

         date_time                                              label  
0     2 tuần trước  {CAMERA#Positive};{

In [55]:
X_train = train_df.iloc[:, 1:-1]
y_train = train_df.iloc[:, -1]

In [57]:
X_test = test_df.iloc[:, 1:-1]
y_test = test_df.iloc[:, -1]

In [59]:
vetorizar = TfidfVectorizer(max_features=3000, max_df=0.85) 

# Lowercase

In [62]:
def lowercase(df):
    df['comment'] = df['comment'].str.lower()

In [64]:
lowercase(train_df)

In [288]:
label = {'BATTERY':[],'CAMERA':[],'DESIGN':[],'FEATURES':[],'GENERAL':[],'OTHERS':[],'PERFORMANCE':[],'PRICE':[],'SCREEN':[],'SER&ACC':[],'STORAGE':[]}
y = y_train.copy()

for i in y_train:
    x = i[:-1].split(";")
    # print(x)
    temp = list(label.keys())
    for j in x:
        j = j.replace('{', '')
        j = j.replace('}', '')
        t = j.split('#')
        
        if t[0] != "OTHERS":
            temp.remove(t[0])
            if t[1] == "Positive":
                label[t[0]].append(1) #nhan xet tich cuc
            elif t[1] == "Neutral":
                label[t[0]].append(0) #nhan xet trung binh 
            else:
                label[t[0]].append(-1) #nhan xet tieu cuc
                
        else:
            temp.remove(t[0])
            label['OTHERS'].append(1) #phan loai other, ko lien quan
            
    for key in temp:
            label[key].append(-2) #Ko de cap den trong phan nhan xet

    # for key in label.keys():
    #     print(len(label[key]))
y = pd.DataFrame(label)
y

,BATTERY,CAMERA,DESIGN,FEATURES,GENERAL,OTHERS,PERFORMANCE,PRICE,SCREEN,SER&ACC,STORAGE
0,1,1,-2,1,1,-2,-2,1,-2,1,-2
1,-1,-2,-2,-2,1,1,-2,-2,-2,-2,-2
2,-2,-2,-2,-1,-2,-2,-2,-2,-2,-2,-2
3,0,-2,-2,-1,0,-2,-2,-2,-2,-2,-2
4,1,-2,-2,-2,-2,-2,1,-2,-2,-1,-2
...,...,...,...,...,...,...,...,...,...,...,...
7781,-1,-2,-2,-1,-2,-2,-1,-2,-2,-2,-2
7782,-2,-2,-2,-1,-2,-2,-1,1,-2,-2,-2
7783,1,-2,-2,-2,1,-2,1,-2,-2,-2,-2
7784,-2,-2,-2,-2,1,-2,-2,-1,-2,-1,-2


In [278]:
vetorizar = TfidfVectorizer(max_features=3000, max_df=0.85)
vetorizar.fit(X_train)
X_train_tfidf = vetorizar.transform(X_train)
X_test_tfidf = vetorizar.transform(X_test)
X_train_tfidf

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [276]:
mlknn_classifier = MLkNN()
mlknn_classifier.fit(X_train_tfidf, y_train)

AttributeError: 'Series' object has no attribute 'getformat'

# Removing Punctuations

In [ ]:
import string
punc = string.punctuation

In [ ]:
def remove_punc(text):
    return text.translate(str.maketrans('', '', punc))

In [ ]:
def final_rmv_punc(df):
    df['comment'] = df['comment'].apply(remove_punc)

In [ ]:
final_rmv_punc(train_df)

In [97]:
train_df.head()

,index,comment,n_star,date_time,label
0,0,mới mua máy này tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,pin kém còn lại miễn chê mua 832019 tình trạng...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,mọi người cập nhật phần mềm lại nó sẽ bớt tốn...,3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,mới mua sài được 1 tháng thấy pin rất trâu sài...,5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# Removing Number?

In [ ]:
def remove_num(df):
    df['comment'] = df['comment'].replace(to_replace=r'\d', value='', regex=True)

In [ ]:
remove_num(train_df)

In [102]:
train_df.head()

,index,comment,n_star,date_time,label
0,0,mới mua máy này tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,pin kém còn lại miễn chê mua 832019 tình trạng...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,mọi người cập nhật phần mềm lại nó sẽ bớt tốn...,3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,mới mua sài được 1 tháng thấy pin rất trâu sài...,5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# Tokenization

In [105]:
!pip install pyvi

In [106]:
from pyvi import ViTokenizer, ViPosTagger

In [107]:
tokenizer = ViTokenizer.tokenize

In [108]:
def tokenize(df):
    df['comment'] = df['comment'].apply(tokenizer)

In [109]:
tokenize(train_df)

In [124]:
train_df.head()

,index,comment,n_star,date_time,label
0,0,mới mua máy này tại thegioididong thốt_nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,pin kém còn lại miễn chê mua 832019 tình_trạng...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,sao lúc gọi điện_thoại màn_hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,mọi người cập_nhật phần_mềm lại nó sẽ bớt tốn ...,3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,mới mua sài được 1 tháng thấy pin rất trâu sài...,5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# POS tagging

In [112]:
tagger = ViPosTagger.postagging

In [113]:
ex = train_df.iloc[0]['comment']

In [122]:
tagger(ex)

(['mới',
  'mua',
  'máy',
  'này',
  'tại',
  'thegioididong',
  'thốt_nốt',
  'cảm_thấy',
  'ok',
  'bin',
  'trâu',
  'chụp',
  'ảnh',
  'đẹp',
  'loa',
  'nghe',
  'to',
  'bắt',
  'wf',
  'khỏe_sóng',
  'ổn_định',
  'giá_thành',
  'vừa',
  'với',
  'túi_tiền',
  'nhân_viên',
  'tư_vấn',
  'nhiệt_tình'],
 array(['R', 'V', 'N', 'P', 'E', 'N', 'N', 'V', 'N', 'V', 'N', 'V', 'N',
        'A', 'N', 'V', 'A', 'V', 'N', 'N', 'A', 'N', 'R', 'E', 'V', 'N',
        'V', 'A'], dtype=object))